In [122]:
from datetime import timedelta
from datetime import datetime
import pandas as pd
from datetime import timezone
import numpy as np
import pickle

In [123]:
import json

config = json.load(open("../configure.json"))
pkls_dir=config['pkls_dir']
start_date = datetime.fromisoformat(config['start_time']).astimezone(tz=timezone.utc).replace(tzinfo=None).timestamp()
end_date = datetime.fromisoformat(config['end_time']).astimezone(tz=timezone.utc).replace(tzinfo=None).timestamp()

In [124]:
apple_news = pd.read_pickle("pkls/apple_news_2gram.pkl", compression='infer')
amazon_news = pd.read_pickle("pkls/amazon_news_2gram.pkl", compression='infer')
apple_chart = pd.read_pickle("pkls/apple_stock_chart.pkl", compression='infer')
amazon_chart = pd.read_pickle("pkls/apple_stock_chart.pkl", compression='infer')

In [125]:
apple_news

,published_time,article_content,word_tokens,features
12869,1.514876e+09,by nigam arora\nthe practical way to take adva...,"[practic, way, take, advantag, januari, effect...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12886,1.514876e+09,here are some things going on today in your wo...,"[go, today, world, tech, bellweth, tech, march...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13804,1.514876e+09,what happened shares of many optical networkin...,"[mani, optic, decemb, accord, data, p, global,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11106,1.514876e+09,"by ciara linnane and tomi kilgore, marketwatch...","[gain, vener, blue, chip, ge, stock, appl, par...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11837,1.514876e+09,by ryan vlastelica\nto simply match the market...,"[simpli, match, market, may, need, take, lot, ...","[0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
73836,1.519896e+09,don't get too discouraged by wednesday's big m...,"[get, wednesday, big, market, jim, told, mad, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
77943,1.519901e+09,san francisco -- music-streaming pioneer spoti...,"[san, francisco, pioneer, attract, new, crowd,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
75095,1.519908e+09,share tweet \nby ben hirschler \nlondon (reute...,"[share, tweet, ben, race, scoop, patient, heal...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
77284,1.519910e+09,stocks won't test the correction lows february...,"[stock, wo, test, correct, februari, kevin, co...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [126]:
amazon_news

,published_time,article_content,word_tokens,features
12869,1.514876e+09,by nigam arora\nthe practical way to take adva...,"[practic, way, take, advantag, januari, effect...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12886,1.514876e+09,here are some things going on today in your wo...,"[go, today, world, tech, bellweth, tech, march...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13804,1.514876e+09,what happened shares of many optical networkin...,"[mani, optic, decemb, accord, data, p, global,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11106,1.514876e+09,"by ciara linnane and tomi kilgore, marketwatch...","[gain, vener, blue, chip, ge, stock, appl, par...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11837,1.514876e+09,by ryan vlastelica\nto simply match the market...,"[simpli, match, market, may, need, take, lot, ...","[0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
73836,1.519896e+09,don't get too discouraged by wednesday's big m...,"[get, wednesday, big, market, jim, told, mad, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
77943,1.519901e+09,san francisco -- music-streaming pioneer spoti...,"[san, francisco, pioneer, attract, new, crowd,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
75095,1.519908e+09,share tweet \nby ben hirschler \nlondon (reute...,"[share, tweet, ben, race, scoop, patient, heal...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
77284,1.519910e+09,stocks won't test the correction lows february...,"[stock, wo, test, correct, februari, kevin, co...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [127]:
amazon_news['trend'] = [0 for i in range(0, len(amazon_news))]
amazon_news

,published_time,article_content,word_tokens,features,trend
12869,1.514876e+09,by nigam arora\nthe practical way to take adva...,"[practic, way, take, advantag, januari, effect...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
12886,1.514876e+09,here are some things going on today in your wo...,"[go, today, world, tech, bellweth, tech, march...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
13804,1.514876e+09,what happened shares of many optical networkin...,"[mani, optic, decemb, accord, data, p, global,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
11106,1.514876e+09,"by ciara linnane and tomi kilgore, marketwatch...","[gain, vener, blue, chip, ge, stock, appl, par...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
11837,1.514876e+09,by ryan vlastelica\nto simply match the market...,"[simpli, match, market, may, need, take, lot, ...","[0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, ...",0
...,...,...,...,...,...
73836,1.519896e+09,don't get too discouraged by wednesday's big m...,"[get, wednesday, big, market, jim, told, mad, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
77943,1.519901e+09,san francisco -- music-streaming pioneer spoti...,"[san, francisco, pioneer, attract, new, crowd,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
75095,1.519908e+09,share tweet \nby ben hirschler \nlondon (reute...,"[share, tweet, ben, race, scoop, patient, heal...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
77284,1.519910e+09,stocks won't test the correction lows february...,"[stock, wo, test, correct, februari, kevin, co...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0


In [128]:
import time, datetime

index = 0
row = []
minutes = 1440

form = "%d-%m-%Y %H:%M"
trends = []


def get_time_to_trend(chart, time_interval):
    time_map = {}
    if minutes == time_interval:
        for chart_index, chart_row in chart.iterrows():
            day = datetime.datetime.utcfromtimestamp(chart_row['timestamp'])
            day = (day - datetime.timedelta(hours=day.hour, minutes=day.minute, seconds=day.second,
                                            microseconds=day.microsecond))
            timestamp = day.strftime(form)
            time_map[timestamp] = chart_row['trends']
    return time_map


time_to_trend = get_time_to_trend(amazon_chart, 1440)

try:
    for index, row in amazon_news.iterrows():
        day = datetime.datetime.utcfromtimestamp(row['published_time'])
        day = (day - datetime.timedelta(hours=day.hour, minutes=day.minute, seconds=day.second,
                                        microseconds=day.microsecond))
        timestamp = day.strftime(form)
        while timestamp not in time_to_trend:
            day += datetime.timedelta(hours=24)
            timestamp = day.strftime(form)
        trends.append(int(time_to_trend[timestamp]))
except Exception as e:
    print(e)
    print("it fails")
    print(row, index)
amazon_news['trend'] = trends

In [129]:
amazon_news

,published_time,article_content,word_tokens,features,trend
12869,1.514876e+09,by nigam arora\nthe practical way to take adva...,"[practic, way, take, advantag, januari, effect...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
12886,1.514876e+09,here are some things going on today in your wo...,"[go, today, world, tech, bellweth, tech, march...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
13804,1.514876e+09,what happened shares of many optical networkin...,"[mani, optic, decemb, accord, data, p, global,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
11106,1.514876e+09,"by ciara linnane and tomi kilgore, marketwatch...","[gain, vener, blue, chip, ge, stock, appl, par...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
11837,1.514876e+09,by ryan vlastelica\nto simply match the market...,"[simpli, match, market, may, need, take, lot, ...","[0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, ...",1
...,...,...,...,...,...
73836,1.519896e+09,don't get too discouraged by wednesday's big m...,"[get, wednesday, big, market, jim, told, mad, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1
77943,1.519901e+09,san francisco -- music-streaming pioneer spoti...,"[san, francisco, pioneer, attract, new, crowd,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1
75095,1.519908e+09,share tweet \nby ben hirschler \nlondon (reute...,"[share, tweet, ben, race, scoop, patient, heal...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1
77284,1.519910e+09,stocks won't test the correction lows february...,"[stock, wo, test, correct, februari, kevin, co...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1


In [130]:
import numpy

amazon_news.drop("word_tokens", axis=1, inplace=True)
amazon_news.drop("article_content", axis=1, inplace=True)
amazon_news.drop("published_time", axis=1, inplace=True)

amazon_news

,features,trend
12869,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
12886,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
13804,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
11106,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
11837,"[0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, ...",1
...,...,...
73836,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1
77943,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1
75095,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1
77284,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1


In [131]:
with open(pkls_dir + '/amazon_data_for_training.pkl', 'wb') as f:
    pickle.dump(amazon_news, f)